In [6]:
import numpy as np
import pandas as pd

In [7]:
def Load_data():
    
    # Training Data
    label_file = open("train-labels.idx1-ubyte_", "rb")
    image_file = open("train-images.idx3-ubyte_", "rb")
    
    label_file.read(8)
    image_file.read(4)
    number = int.from_bytes(image_file.read(4), byteorder='big')
    row = int.from_bytes(image_file.read(4), byteorder='big')
    col = int.from_bytes(image_file.read(4), byteorder='big')
    
    # Initialze data structure
    training_label = np.zeros(number, dtype=int)
    training_data = np.zeros((number, row, col), dtype=int)
    
    # Load training data
    for i in range(number):
        training_label[i] = label_file.read(1)[0]
        for j in range(row):
            for k in range(col):
                training_data[i][j][k] = image_file.read(1)[0]
                
    label_file.close()
    image_file.close()
    return training_label, training_data

In [8]:
training_label, training_data = Load_data()

In [9]:
class EM_algo:
    def __init__(self):
        self.pi = [0.1 for _ in range(10)]
        self.theta = [np.random.rand(784) for _ in range(10)]
        self.epsilon = 1e-10
    
    def process_img(self, data):
        return (data >= 128).astype(int).reshape(-1, 784)
  
    def optimize(self):
        # E-step
        log_prob = np.zeros([60000, 10])
        binary_img = self.process_img(training_data)
        iter_n = 0
        
        while iter_n < 100:
            for j in range(10):
                theta_safe = np.clip(self.theta[j].reshape(-1), self.epsilon, 1-self.epsilon)
                log_prob[:, j] = np.sum(
                    binary_img * np.log(theta_safe) + (1 - binary_img) * np.log(1 - theta_safe), axis=1
                ) + np.log(self.pi[j] + self.epsilon)
            log_likelihood_max = np.max(log_prob, axis=1, keepdims=True)
            likelihood = np.exp(log_prob - log_likelihood_max)
            responsibility = likelihood / (np.sum(likelihood, axis=1, keepdims=True) + self.epsilon)

            update_pi = np.mean(responsibility, axis=0)
            update_theta = responsibility.T @ binary_img / (np.sum(responsibility, axis=0).reshape(-1, 1) + self.epsilon)
            
            self.print_imagination_of_numbers(update_theta)
            print(f"No. of Iteration:  {iter_n}, Difference: {np.linalg.norm(update_pi - self.pi) + np.linalg.norm(update_theta - np.array(self.theta).reshape(10, 784))}")

            if np.linalg.norm(update_pi - self.pi) < 1e-2 or np.linalg.norm(update_theta - np.array(self.theta).reshape(10, 784)) < 1e-2:
                break
                
            self.pi = update_pi
            self.theta = update_theta
            iter_n += 1

        return self.pi, self.theta, responsibility
    
    def match_clusters_to_labels(self, responsibility, labels):
        """
        將 EM 演算法的聚類結果對應到真實的標籤。
        
        :param responsibility: (N, 10) 的 `responsibility` 矩陣
        :param labels: 真實的標籤數組
        :return: 一個包含對應關係的字典，例如 {0: 3, 1: 0, ...}，表示聚類 0 對應標籤 3
        """
        cluster_label_counts = np.zeros((10, 10), dtype=int)

        for i in range(len(labels)):
            cluster = np.argmax(responsibility[i])  # 找到每張圖片的聚類
            label = labels[i]
            cluster_label_counts[cluster, label] += 1

        # 進行貪心匹配
        cluster_to_label = {}
        used_labels = set()  # 用於記錄已分配的標籤

        for cluster in range(10):
            # 找到該聚類中最常見的真實標籤，並確保不重複使用標籤
            sorted_labels = np.argsort(cluster_label_counts[cluster])[::-1]
            for label in sorted_labels:
                if label not in used_labels:
                    cluster_to_label[cluster] = label
                    used_labels.add(label)
                    break

        return cluster_to_label

    def confusion_matrix(self, pred, y):
        confusion_matrices = {}
        sensitivities = {}
        specificities = {}
        
        for j in range(10):
            pred_binary = (pred == j).astype(int)
            y_binary = (y == j).astype(int)

            TP = np.sum((pred_binary == 1) & (y_binary == 1))
            FP = np.sum((pred_binary == 1) & (y_binary == 0))
            TN = np.sum((pred_binary == 0) & (y_binary == 0))
            FN = np.sum((pred_binary == 0) & (y_binary == 1))

            confusion = pd.DataFrame(
                {
                    f'Predict number {j}': [TP, FP],
                    f'Predict not number {j}': [FN, TN]
                },
                index=[f"Is number {j}", f"Isn't number {j}"]
            )

            sensitivity = TP / (TP + FN) 
            specificity = TN / (FP + TN) 

            confusion_matrices[f"Cluster_{j}"] = confusion
            sensitivities[f"Cluster_{j}"] = sensitivity
            specificities[f"Cluster_{j}"] = specificity
        
        return confusion_matrices, sensitivities, specificities
    
    def print_imagination_of_numbers(self, theta):
        for j in range(10):
            print(f"Class {j}:")
            imagination = (theta[j] > 0.5).astype(int).reshape((28, 28))
            for row in imagination:
                print(" ".join(str(pixel) for pixel in row)) 
            print("\n" + "-" * 50)
            
    def result(self, labels):
        _, _, responsibility = self.optimize()
        self.print_imagination_of_numbers(self.theta)
            
        preds = np.argmax(responsibility, axis=1)
        
        cluster_to_label = self.match_clusters_to_labels(responsibility, labels)
        mapped_preds = np.vectorize(cluster_to_label.get)(preds)
        
        conf, sens, spec = self.confusion_matrix(mapped_preds, labels)
        
        for j in range(10):
            print(f'Confusion Matrix for Cluster {j}:')
            print(conf[f"Cluster_{j}"])
            print(f'Sensitivity (Successfully predict cluster {j}): {sens[f"Cluster_{j}"]:.4f}')
            print(f'Specificity (Successfully predict not cluster {j}): {spec[f"Cluster_{j}"]:.4f}')
            
            if j != 9:
                print('-' * 70)
            
if __name__ == "__main__":
    em = EM_algo()
    em.result(training_label)

Class 0:
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1